# <center>PROJECT SANDBOX</center>

## Documentation
The aim of this notebook is to provide a simple sandbox to test different NN architectures for the project. , here is a doc about the functions imported from `scripts` folder : 

- **`prepare_dataset(device,ratio=0.5,shuffle_ctx=False)`** :
    - **Input**:
        - device : a torch.device object
        - ratio : a float ratio between 0 and 1 that determines the average proportion of modern english verses in the data loader
        - shuffle_ctx : if `True`, shuffle the contexts within a Batch so that half of the `x_1` elements has a wrong context `ctx_1`. Useful to train the context recognizer model.
    - **Return** :
        - a torch Dataset | class : Shakespeare inherited from torch.utils.data.Dataset
        - a python word dictionary (aka tokenizer) | class : dict
    - **Tensors returned when loaded in the dataloader**:
        - x_1 : input verse (modern / shakespearian)
        - x_2 : output verse (modern / shakespearian)

        - ctx_1 = context of the input verse
        - ctx_2 = context of the output verse

        - len_x : length of the input verse
        - len_y : length of the output verse

        - len_ctx_x : length of the input verse context
        - len_ctx_y : length of the output verse context

        - label : label of the input verse (0 : modern, 1 : shakespearian)
        - label_ctx : label of the context (0 : wrong context, 1 : right context)
- **`string2code(string,dict)`** : 
    - **Input**:
        - string : a sentence
        - dict : a tokenizer
    - **Return** :
        - a torch Longtensor (sentence tokenized)
- **`code2string(torch.Longtensor,dict)`** : 
    - **Input**:
        - torch.Longtensor : a sentence tokenized
        - dict : a tokenizer
    - **Return** :
        - a string sentence

## Importing packages

In [1]:
from scripts.data_builders.prepare_dataset import prepare_dataset_ctx,string2code,code2string,assemble

import torch
import torchvision.datasets as datasets
import torch.nn.functional as F
from torch import nn
from torch import optim
from torch.utils.tensorboard import SummaryWriter
import math
from torch.nn import BCELoss,CrossEntropyLoss
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import pickle
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device = ",device)

device =  cuda


## Preprocessing data

In [2]:
train_data, dict_words = prepare_dataset_ctx("data/shakespeare.csv",device,ratio=0.5,shuffle_ctx=True) #check with shift+tab to look at the data structure
batch_size = 8
dict_token = {b:a for a,b in dict_words.items()} #dict for code2string

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                           shuffle=True,collate_fn=train_data.collate)
dict_size = len(dict_words)
d_embedding = 300 #cf. paper Y.Kim 2014 Convolutional Neural Networks for Sentence Classification

print("- dict size : ",dict_size)

Loading ...
- Shakespeare context dataset length :  21079
- Corrupted samples (ignored) :  0
- dict size :  17513


## Designing NN model

# Model 1 : CNN

In [3]:
class CoherenceClassifier(torch.nn.Module):
    def __init__(self,dict_size=dict_size,d_embedding=300):
        super().__init__()
        self.embed_layer=torch.nn.Embedding(dict_size+1,d_embedding,padding_idx=dict_size)

        self.conv_1 = torch.nn.Conv1d(d_embedding,3,kernel_size = 3, stride = 1)
        self.max_pool = torch.nn.MaxPool1d(3,2)
        self.relu = torch.nn.ReLU()
        self.linear = torch.nn.Linear(3,1)
    
    def forward(self,x):
        x = self.embed_layer(x)
        x = self.conv_1( x.transpose(1,2))
        x = self.max_pool( x )
        x = self.relu( x )
        x = torch.max( x , 2 )[0]
        x = torch.sigmoid(self.linear(x))
        return x

In [4]:
epochs = 100
model = CoherenceClassifier().to(device)
optimizer = optim.Adam(params=model.parameters(),lr=0.01)
loss_func = BCELoss()
n = len(train_data.x) // batch_size

In [5]:
for epoch in range(epochs):
    total_loss = 0
    i = 0
    for _,_ , ctx,_ , _,_ , len_ctx,_, _,label_ctx in train_loader:
        i+=1
        optimizer.zero_grad()
        
        #CNN
        ctx = model.forward(ctx).reshape(-1) #CNN architecture
        
        loss = loss_func( ctx , label_ctx.float() )
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
        #Vizualization

    print('-' * 35)
    print('| epoch {:3d} | '
          'lr {:02.2f} | '
          'loss {:5.2f}'.format(
            epoch+1, optimizer.state_dict()["param_groups"][0]["lr"],
            round(total_loss,2)))

ValueError: not enough values to unpack (expected 10, got 4)

# Model 2 : LSTM 

In [ ]:
class CoherenceClassifier(torch.nn.Module):
    def __init__(self,dict_size=dict_size,d_embedding=300,d_hidden=100):
        super().__init__()
        self.d_hidden = d_hidden
        self.embedding = nn.Embedding(dict_size+1,d_embedding,padding_idx=dict_size)
        self.lstm = nn.LSTM(d_embedding,self.d_hidden,dropout=0.,num_layers=1,bidirectional=False)
        #self.bn0 = nn.BatchNorm1d(self.d_hidden)
        self.linear1 = torch.nn.Linear(self.d_hidden,1)
    
    def forward(self,x,len_x):
        x = self.embedding(x)
        x = pack_padded_sequence(x.permute(1,0,2),len_x,enforce_sorted=False)
        _,x = self.lstm(x)
        x = x[0].reshape(-1,self.d_hidden)
        #x = self.bn0(x)
        x = torch.sigmoid( self.linear1(x) ).reshape(-1)
        return x

In [ ]:
epochs = 100
model = CoherenceClassifier().to(device)
optimizer = optim.Adam(params=model.parameters(),lr=0.01)
loss_func = BCELoss()
n = len(train_data.x) // batch_size

In [ ]:
for epoch in range(epochs):
    total_loss = 0
    i = 0
    for _,_ , ctx,_ , _,_ , len_ctx,_, _,label_ctx in train_loader:
        i+=1
        optimizer.zero_grad()
        
        #LSTM
        ctx = model.forward(ctx,len_ctx) #LSTM architecture
        
        loss = loss_func( ctx , label_ctx.float() )
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    print('-' * 35)
    print('| epoch {:3d} | '
          'lr {:02.2f} | '
          'loss {:5.2f}'.format(
            epoch+1, optimizer.state_dict()["param_groups"][0]["lr"],
            round(total_loss,2)))

# Model 3 : Transformers

In [ ]:
class CoherenceClassifier(nn.Module):

    def __init__(self,dict_size=dict_size, d_embedding=300,  dropout=0.1):
        super(CoherenceClassifier, self).__init__()
        
        self.embedding = nn.Embedding(dict_size+1,d_embedding,padding_idx=dict_size)
        self.pos_encoder = PositionalEncoding(d_embedding, dropout)
        encoder_layers = TransformerEncoderLayer(d_model=d_embedding, nhead = 4,dropout=dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, num_layers=4)
        
        self.decoder = nn.Linear(d_embedding, 2 )

    def forward(self, x):

        x = self.embedding( x )
        #x = self.pos_encoder( x )
        x = self.transformer_encoder( x )
        x = torch.softmax(torch.tanh(self.decoder( x )),1)
        return x
    
    
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=50):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x * math.sqrt(self.d_model)
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [ ]:
epochs = 100
model = CoherenceClassifier().to(device)
optimizer = optim.Adam(params=model.parameters(),lr=0.01)
loss_func = CrossEntropyLoss()
n = len(train_data.x) // batch_size

In [ ]:
for epoch in range(epochs):
    total_loss = 0
    i = 0
    for _,_ , ctx,_ , _,_ , len_ctx,_, _,label_ctx in train_loader:
        i+=1
        optimizer.zero_grad()
        
        #Transformer
        ctx = model.forward(ctx) #Transformer architecture
        y = torch.cat([label_ctx.reshape(-1,1),1-label_ctx.reshape(-1,1)],dim=1) # Transformer architecture
        
        loss = loss_func( ctx , y )
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    print('-' * 35)
    print('| epoch {:3d} | '
          'lr {:02.2f} | '
          'loss {:5.2f}'.format(
            epoch+1, optimizer.state_dict()["param_groups"][0]["lr"],
            round(total_loss,2)))

# Model 4 :  Pre-trained model

In [3]:
epochs = 100

#Load model and plug our Embedding in, and freeze Embedding
model = torch.hub.load('huggingface/pytorch-transformers', 'modelForSequenceClassification', 'bert-base-uncased').to(device)
model.bert.embeddings.word_embeddings = torch.load("data/models/embeddings/v0").to(device)
for param in model.bert.embeddings.word_embeddings.parameters():
    param.requires_grad = False
    
    
model.train()

optimizer = optim.Adam(params=model.parameters(),lr=0.01)
loss_func = CrossEntropyLoss()
n = len(train_data.x) // batch_size

Using cache found in /home/jb/.cache/torch/hub/huggingface_pytorch-transformers_master


In [4]:
for epoch in range(epochs):
    total_loss,total_accuracy = 0,0
    i = 0
    for ctx,pos_token,pos_ctx,label in train_loader:
        i+=1
        optimizer.zero_grad()
        
        #pre-trained BERT
        ctx = model.forward(input_ids=ctx,
                           token_type_ids=pos_ctx,
                            position_ids=pos_token)[0]
        
        loss = loss_func( ctx , label )
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        accuracy = 1 - (ctx.argmax(dim=1) - label).abs().sum().item() / ctx.shape[0]
        total_accuracy += accuracy
        
        print('| epoch {:3d} | {:5d}/{:5d} batches | '
              'loss {:5.2f} | total loss {:5.2f} | '
              'accuracy {:5.2f} | total accuracy {:5.2f}'.format(
                epoch+1, i, n,loss.item(),total_loss/(i+n*epoch),accuracy,total_accuracy/(i+n*epoch)))
    print('-' * 35)
    print("Epoch ",epoch,"\t",round(total_loss / n,2))

| epoch   1 |     1/ 2634 batches | loss  1.10 | total loss  1.10 | accuracy  0.12 | total accuracy  0.12
| epoch   1 |     2/ 2634 batches | loss  4.02 | total loss  2.56 | accuracy  0.25 | total accuracy  0.19
| epoch   1 |     3/ 2634 batches | loss  1.40 | total loss  2.17 | accuracy  0.88 | total accuracy  0.42
| epoch   1 |     4/ 2634 batches | loss  7.42 | total loss  3.49 | accuracy  0.25 | total accuracy  0.38
| epoch   1 |     5/ 2634 batches | loss  1.75 | total loss  3.14 | accuracy  0.62 | total accuracy  0.42
| epoch   1 |     6/ 2634 batches | loss  0.82 | total loss  2.75 | accuracy  0.75 | total accuracy  0.48
| epoch   1 |     7/ 2634 batches | loss  1.81 | total loss  2.62 | accuracy  0.38 | total accuracy  0.46
| epoch   1 |     8/ 2634 batches | loss  2.12 | total loss  2.55 | accuracy  0.38 | total accuracy  0.45
| epoch   1 |     9/ 2634 batches | loss  1.04 | total loss  2.39 | accuracy  0.38 | total accuracy  0.44
| epoch   1 |    10/ 2634 batches | loss  1.07

| epoch   1 |    79/ 2634 batches | loss  0.81 | total loss  1.30 | accuracy  0.62 | total accuracy  0.51
| epoch   1 |    80/ 2634 batches | loss  2.51 | total loss  1.31 | accuracy  0.50 | total accuracy  0.51
| epoch   1 |    81/ 2634 batches | loss  3.50 | total loss  1.34 | accuracy  0.38 | total accuracy  0.51
| epoch   1 |    82/ 2634 batches | loss  1.89 | total loss  1.35 | accuracy  0.50 | total accuracy  0.51
| epoch   1 |    83/ 2634 batches | loss  0.77 | total loss  1.34 | accuracy  0.62 | total accuracy  0.51
| epoch   1 |    84/ 2634 batches | loss  1.18 | total loss  1.34 | accuracy  0.75 | total accuracy  0.51
| epoch   1 |    85/ 2634 batches | loss  0.00 | total loss  1.32 | accuracy  1.00 | total accuracy  0.52
| epoch   1 |    86/ 2634 batches | loss  3.58 | total loss  1.35 | accuracy  0.62 | total accuracy  0.52
| epoch   1 |    87/ 2634 batches | loss  7.55 | total loss  1.42 | accuracy  0.25 | total accuracy  0.52
| epoch   1 |    88/ 2634 batches | loss  4.11

| epoch   1 |   158/ 2634 batches | loss  0.66 | total loss  1.41 | accuracy  0.50 | total accuracy  0.51
| epoch   1 |   159/ 2634 batches | loss  0.71 | total loss  1.41 | accuracy  0.50 | total accuracy  0.51
| epoch   1 |   160/ 2634 batches | loss  0.86 | total loss  1.41 | accuracy  0.50 | total accuracy  0.51
| epoch   1 |   161/ 2634 batches | loss  0.78 | total loss  1.40 | accuracy  0.38 | total accuracy  0.50
| epoch   1 |   162/ 2634 batches | loss  0.42 | total loss  1.40 | accuracy  0.88 | total accuracy  0.51
| epoch   1 |   163/ 2634 batches | loss  0.62 | total loss  1.39 | accuracy  0.75 | total accuracy  0.51
| epoch   1 |   164/ 2634 batches | loss  1.02 | total loss  1.39 | accuracy  0.62 | total accuracy  0.51
| epoch   1 |   165/ 2634 batches | loss  0.78 | total loss  1.39 | accuracy  0.62 | total accuracy  0.51
| epoch   1 |   166/ 2634 batches | loss  0.99 | total loss  1.38 | accuracy  0.38 | total accuracy  0.51
| epoch   1 |   167/ 2634 batches | loss  0.87

| epoch   1 |   236/ 2634 batches | loss  0.74 | total loss  1.31 | accuracy  0.62 | total accuracy  0.50
| epoch   1 |   237/ 2634 batches | loss  0.71 | total loss  1.31 | accuracy  0.62 | total accuracy  0.50
| epoch   1 |   238/ 2634 batches | loss  0.87 | total loss  1.31 | accuracy  0.50 | total accuracy  0.50
| epoch   1 |   239/ 2634 batches | loss  0.86 | total loss  1.31 | accuracy  0.62 | total accuracy  0.50
| epoch   1 |   240/ 2634 batches | loss  0.93 | total loss  1.30 | accuracy  0.62 | total accuracy  0.51
| epoch   1 |   241/ 2634 batches | loss  0.50 | total loss  1.30 | accuracy  0.75 | total accuracy  0.51
| epoch   1 |   242/ 2634 batches | loss  0.62 | total loss  1.30 | accuracy  0.62 | total accuracy  0.51
| epoch   1 |   243/ 2634 batches | loss  0.77 | total loss  1.30 | accuracy  0.62 | total accuracy  0.51
| epoch   1 |   244/ 2634 batches | loss  0.60 | total loss  1.29 | accuracy  0.75 | total accuracy  0.51
| epoch   1 |   245/ 2634 batches | loss  1.54

| epoch   1 |   314/ 2634 batches | loss  1.46 | total loss  1.23 | accuracy  0.38 | total accuracy  0.50
| epoch   1 |   315/ 2634 batches | loss  0.74 | total loss  1.23 | accuracy  0.75 | total accuracy  0.50
| epoch   1 |   316/ 2634 batches | loss  0.68 | total loss  1.23 | accuracy  0.62 | total accuracy  0.50
| epoch   1 |   317/ 2634 batches | loss  0.90 | total loss  1.23 | accuracy  0.38 | total accuracy  0.50
| epoch   1 |   318/ 2634 batches | loss  0.80 | total loss  1.23 | accuracy  0.62 | total accuracy  0.50
| epoch   1 |   319/ 2634 batches | loss  0.68 | total loss  1.23 | accuracy  0.50 | total accuracy  0.50
| epoch   1 |   320/ 2634 batches | loss  0.66 | total loss  1.22 | accuracy  0.75 | total accuracy  0.51
| epoch   1 |   321/ 2634 batches | loss  0.91 | total loss  1.22 | accuracy  0.38 | total accuracy  0.51
| epoch   1 |   322/ 2634 batches | loss  0.80 | total loss  1.22 | accuracy  0.50 | total accuracy  0.51
| epoch   1 |   323/ 2634 batches | loss  0.65

...

| epoch   2 |  1594/ 2634 batches | loss  1.20 | total loss  0.51 | accuracy  0.25 | total accuracy  0.19
| epoch   2 |  1595/ 2634 batches | loss  0.82 | total loss  0.51 | accuracy  0.75 | total accuracy  0.19
| epoch   2 |  1596/ 2634 batches | loss  3.19 | total loss  0.51 | accuracy  0.25 | total accuracy  0.19
| epoch   2 |  1597/ 2634 batches | loss  1.97 | total loss  0.51 | accuracy  0.50 | total accuracy  0.19
| epoch   2 |  1598/ 2634 batches | loss  0.81 | total loss  0.51 | accuracy  0.38 | total accuracy  0.19
| epoch   2 |  1599/ 2634 batches | loss  2.01 | total loss  0.51 | accuracy  0.38 | total accuracy  0.19
| epoch   2 |  1600/ 2634 batches | loss  0.93 | total loss  0.51 | accuracy  0.62 | total accuracy  0.19
| epoch   2 |  1601/ 2634 batches | loss  0.93 | total loss  0.51 | accuracy  0.38 | total accuracy  0.19
| epoch   2 |  1602/ 2634 batches | loss  0.87 | total loss  0.51 | accuracy  0.50 | total accuracy  0.19
| epoch   2 |  1603/ 2634 batches | loss  1.56

| epoch   2 |  1673/ 2634 batches | loss  0.57 | total loss  0.52 | accuracy  0.62 | total accuracy  0.20
| epoch   2 |  1674/ 2634 batches | loss  0.90 | total loss  0.52 | accuracy  0.38 | total accuracy  0.20
| epoch   2 |  1675/ 2634 batches | loss  1.42 | total loss  0.52 | accuracy  0.00 | total accuracy  0.20
| epoch   2 |  1676/ 2634 batches | loss  0.86 | total loss  0.52 | accuracy  0.62 | total accuracy  0.20
| epoch   2 |  1677/ 2634 batches | loss  1.09 | total loss  0.52 | accuracy  0.62 | total accuracy  0.20
| epoch   2 |  1678/ 2634 batches | loss  1.74 | total loss  0.52 | accuracy  0.50 | total accuracy  0.20
| epoch   2 |  1679/ 2634 batches | loss  0.91 | total loss  0.52 | accuracy  0.62 | total accuracy  0.20
| epoch   2 |  1680/ 2634 batches | loss  0.89 | total loss  0.52 | accuracy  0.25 | total accuracy  0.20
| epoch   2 |  1681/ 2634 batches | loss  1.08 | total loss  0.52 | accuracy  0.62 | total accuracy  0.20
| epoch   2 |  1682/ 2634 batches | loss  1.16

| epoch   2 |  1752/ 2634 batches | loss  0.70 | total loss  0.53 | accuracy  0.75 | total accuracy  0.20
| epoch   2 |  1753/ 2634 batches | loss  0.49 | total loss  0.53 | accuracy  0.75 | total accuracy  0.20
| epoch   2 |  1754/ 2634 batches | loss  1.88 | total loss  0.53 | accuracy  0.25 | total accuracy  0.20
| epoch   2 |  1755/ 2634 batches | loss  0.72 | total loss  0.53 | accuracy  0.50 | total accuracy  0.20
| epoch   2 |  1756/ 2634 batches | loss  0.39 | total loss  0.53 | accuracy  0.88 | total accuracy  0.20
| epoch   2 |  1757/ 2634 batches | loss  1.19 | total loss  0.53 | accuracy  0.62 | total accuracy  0.20
| epoch   2 |  1758/ 2634 batches | loss  1.00 | total loss  0.53 | accuracy  0.75 | total accuracy  0.20
| epoch   2 |  1759/ 2634 batches | loss  1.75 | total loss  0.53 | accuracy  0.25 | total accuracy  0.20
| epoch   2 |  1760/ 2634 batches | loss  1.01 | total loss  0.53 | accuracy  0.38 | total accuracy  0.20
| epoch   2 |  1761/ 2634 batches | loss  0.47

| epoch   2 |  1831/ 2634 batches | loss  0.66 | total loss  0.55 | accuracy  0.62 | total accuracy  0.21
| epoch   2 |  1832/ 2634 batches | loss  0.72 | total loss  0.55 | accuracy  0.50 | total accuracy  0.21
| epoch   2 |  1833/ 2634 batches | loss  0.83 | total loss  0.55 | accuracy  0.62 | total accuracy  0.21
| epoch   2 |  1834/ 2634 batches | loss  0.92 | total loss  0.55 | accuracy  0.38 | total accuracy  0.21
| epoch   2 |  1835/ 2634 batches | loss  0.79 | total loss  0.55 | accuracy  0.50 | total accuracy  0.21
| epoch   2 |  1836/ 2634 batches | loss  0.95 | total loss  0.55 | accuracy  0.38 | total accuracy  0.21
| epoch   2 |  1837/ 2634 batches | loss  0.74 | total loss  0.55 | accuracy  0.50 | total accuracy  0.21
| epoch   2 |  1838/ 2634 batches | loss  0.69 | total loss  0.55 | accuracy  0.62 | total accuracy  0.21
| epoch   2 |  1839/ 2634 batches | loss  1.21 | total loss  0.55 | accuracy  0.50 | total accuracy  0.21
| epoch   2 |  1840/ 2634 batches | loss  1.54

KeyboardInterrupt: 

# Draft

In [ ]:
for ctx,pos_token,pos_ctx,label in train_loader:
    print(code2string(ctx[0],dict_token))
    print(pos_ctx[0])
    print(label[0].item())
    break

In [ ]:
torch.LongTensor([0])

In [6]:
50 % 100

50

219

0.25

In [11]:
ctx

tensor([[-0.1767,  0.7985],
        [-0.3940,  0.8467],
        [-0.4016,  0.7627],
        [-0.4417,  0.9521],
        [-0.2279,  0.7989],
        [-0.0924,  0.3308],
        [-0.2382,  0.8198],
        [-0.6877,  1.0233]], device='cuda:0', grad_fn=<AddmmBackward>)

In [12]:
label

tensor([1, 0, 0, 0, 1, 0, 0, 0], device='cuda:0')